# Clean_Data_GOOD

## Imported libraries

In [1]:
import requests #To send requests to an API
import json #To read the content of the returned information of the API


import pandas as pd #To work with dataframes
import numpy as np #To work with np.arrays

## Reading the data

In [2]:
df = pd.read_csv('./datasets/data_100000.csv')

## Preprocessing the data

#### Dropping columns with excessive (or duplicate) information

In [3]:
"""
information about the dropped columns: 
- contributing_factor_vehicle_1 to contributing_factor_vehicle_5: Not neccessary to know the most dangerous streets
- vehicle_type_code1, vehicle_type_code2: Not neccessary to know the most dangerous streets
- vehicle_type_code_3 tot vehicle_type_code_5: Not neccessary to know the most dangerous streets
- location: Because the latitude and longitude are already available, this information is excessive

"""

df = df.drop(['vehicle_type_code1',
         'vehicle_type_code2',
         'vehicle_type_code_3',
         'vehicle_type_code_4',
         'vehicle_type_code_5',
        'contributing_factor_vehicle_1',
         'contributing_factor_vehicle_2',
         'contributing_factor_vehicle_3',
        'contributing_factor_vehicle_4',
        'contributing_factor_vehicle_5',
        'location'], axis=1)

#### Replacing empty values("" of null) with "Information not available" string

In [4]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.fillna("Information not available")

#### Filling on_street_name with the cross_street_name (stripped from the address number)

In [5]:
df.cross_street_name = df.cross_street_name.astype('string')
for i in range(0,df.shape[0]):
    if df.cross_street_name.iloc[i] != "Information not available":
        sections = df.cross_street_name.iloc[i].split(" ").copy()
        sections.remove(sections[0])
        df.on_street_name.iloc[i] = ' '.join(map(str, sections))
    else:
        continue

C:\Users\VervoortLaptop\anaconda3\envs\Standard_Work_Env\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


#### Dropping the cross_street_name column

In [6]:
df = df.drop(['cross_street_name'], axis=1)

#### Getting the missing zipcodes and their boroughs

In [ ]:
for i in range(0,df.shape[0]):
    if df.zip_code.iloc[i] == "INFORMATION NOT AVAILABLE":
    
        longitude = df.longitude.iloc[i]
        latitude = df.latitude.iloc[i]
        try:
        address = requests.get(f"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&featureTypes=&location={longitude}%2C{latitude}").json()
        
        sections = address['address']['ShortLabel'].split(" ")
        sections.remove(sections[0])
        try: 
            if sections[-1] == 'Ave':
                sections[-1] = 'Avenue'
            elif sections[-1] == 'Rd':
                sections[-1] = 'Road'
            elif sections[-1] == 'St':
                sections[-1] = 'Street'
        except IndexError:
            continue
        
        df.on_street_name[i] = ' '.join(map(str, sections))
        
        
        except KeyError:
            continue


#### Removing all rows that contain "Information not available" string within column on_street_name

In [7]:
df = df[df['on_street_name'] != "Information not available"]

#### Changing the dataframe to dtype: "string"

In [8]:
df = df.applymap(str)

#### Removing all the excessive white spaces

In [9]:
df = df.applymap(str.strip).rename(columns=str.strip)

#### Changing the dtypes of each column to the correct form and consolidate them if necessary

In [10]:
df.crash_date = df.crash_date.astype('datetime64')

df.on_street_name = df.on_street_name.astype('string').str.upper()
df.off_street_name = df.off_street_name.astype('string').str.upper()

df.zip_code = df.zip_code.str.rstrip(".0")

df.number_of_persons_killed = df.number_of_persons_killed.astype('int8')
df.number_of_persons_injured = df.number_of_persons_injured.astype('int8')
df.number_of_pedestrians_killed = df.number_of_pedestrians_killed.astype('int8')
df.number_of_pedestrians_injured = df.number_of_pedestrians_injured.astype('int8')
df.number_of_cyclist_killed = df.number_of_cyclist_killed.astype('int8')
df.number_of_cyclist_injured = df.number_of_cyclist_injured.astype('int8')
df.number_of_motorist_killed = df.number_of_motorist_killed.astype('int8')
df.number_of_motorist_injured = df.number_of_motorist_injured.astype('int8')

#### Checking the zip_code column for right amount of digits

In [11]:
df["zip_code"] = df["zip_code"].replace(r'^(\d){0,4}$', value = "Information not available", regex=True)
df["zip_code"] = df["zip_code"].replace(r'^(\d){6,10}$',  value = "Information not available", regex=True)

#### Setting collision_id as index

In [12]:
df = df.set_index("collision_id")

#### Replacing all ""Information not available"" string values with "INFORMATION NOT AVAILABLE" string values

In [13]:
df = df.replace(["Information not available"], ["INFORMATION NOT AVAILABLE"])

#### Getting the missing zipcodes and their boroughs

In [16]:
for i in range(0,df.shape[0]):
    if df.zip_code.iloc[i] == "INFORMATION NOT AVAILABLE":
    
        longitude = df.longitude.iloc[i]
        latitude = df.latitude.iloc[i]
        try:
            address = requests.get(f"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&featureTypes=&location={longitude}%2C{latitude}").json()
    
            df.zip_code.iloc[i] = address['address']['Postal']
        
            if df.borough.iloc[i] == "INFORMATION NOT AVAILABLE":
                
                df.borough.iloc[i] = address['address']['District']
            else:
                continue
        
        
        except KeyError:
            continue

KeyboardInterrupt: 

#### Getting the missing borough values and their zipcodes

In [ ]:
for i in range(0,df.shape[0]):
    if df.borough.iloc[i] == "INFORMATION NOT AVAILABLE":
    
        longitude = df.longitude.iloc[i]
        latitude = df.latitude.iloc[i]
        try:
            address = requests.get(f"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&featureTypes=&location={longitude}%2C{latitude}").json()
    
            df.borough.iloc[i] = address['address']['District']
        
            if df.zip_code.iloc[i] == "INFORMATION NOT AVAILABLE":
                
                df.zip_code.iloc[i] = address['address']['Postal']
            else:
                continue
        
        
        except KeyError:
            continue

## Saving the preprocessed data as .csv file

In [333]:
df.to_csv("Clean_data_GOOD.csv")